# Implementing a Route Planner
In this project you will use A\* search to implement a "Google-maps" style route planning algorithm.

In [1]:
# Run this cell first!

from helpers import Map, load_map, show_map
from student_code import shortest_path

%load_ext autoreload
%autoreload 2

### Map Basics

In [2]:
map_10 = load_map('map-10.pickle')
show_map(map_10)

The map above (run the code cell if you don't see it) shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [3]:
map_10.intersections

{0: [0.7798606835438107, 0.6922727646627362],
 1: [0.7647837074641568, 0.3252670836724646],
 2: [0.7155217893995438, 0.20026498027300055],
 3: [0.7076566826610747, 0.3278339270610988],
 4: [0.8325506249953353, 0.02310946309985762],
 5: [0.49016747075266875, 0.5464878695400415],
 6: [0.8820353070895344, 0.6791919587749445],
 7: [0.46247219371675075, 0.6258061621642713],
 8: [0.11622158839385677, 0.11236327488812581],
 9: [0.1285377678230034, 0.3285840695698353]}

**Roads**

The `roads` property is a list where, if `i` is an intersection, `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [4]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 

[7, 6, 5]

In [5]:
# This shows the full connectivity of the map
map_10.roads

[[7, 6, 5],
 [4, 3, 2],
 [4, 3, 1],
 [5, 4, 1, 2],
 [1, 2, 3],
 [7, 0, 3],
 [0],
 [0, 5],
 [9],
 [8]]

In [6]:
# map_40 is a bigger map than map_10
map_40 = load_map('map-40.pickle')
show_map(map_40)

### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizaing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [7]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5,16,37,12,34])

### Writing your algorithm
You should open the file `student_code.py` in another tab and work on your algorithm there. Do that by selecting `File > Open` and then selecting the appropriate file.

The algorithm you write will be responsible for generating a `path` like the one passed into `show_map` above. In fact, when called with the same map, start and goal, as above you algorithm should produce the path `[5, 16, 37, 12, 34]`

```bash
> shortest_path(map_40, 5, 34)
[5, 16, 37, 12, 34]
```

In [11]:
import numpy as np
def dist(pointA, pointB):  #pointA == [0.7798606835438107, 0.6922727646627362], pointB == [0.7647837074641568, 0.3252670836724646]
    return np.sqrt((pointA[0]-pointB[0])*(pointA[0]-pointB[0]) + (pointA[1]-pointB[1])*(pointA[1]-pointB[1]))

#case1: if the goal is unreachable, return -1
#case2: when a path has no unexplored new neighbor but not reach the goal,delete the path. 
        #definition of explored node: a node was a frontier and has been searched it's neighbors
#case3: after reach the goal with a path, do not stop, continue search if there is shorther total estimated distance in the path list

def shortest_path(M,start,goal):
    print("shortest path called")
    nodes = M.intersections  # sorten the name for convinience, nodes includes (x, y) coordinates set. e.g. {0:[x0,y0],1:[x1,y1]}
    if start == goal: # edge case: the start is already the goal
        return [goal]
    
    sol_list = {"path":[[start]], "path_cost":[0], "total_est_cost":[0]}  # save pathes list
                #This list will be refreshed by every newly explored neighbor of a frontier.
                # a frontier: The nearest node to the goal in a path
                # e.g. path: [[1,2,3],[1,5,6],[1,3,7]], path_cost:[400,500,600], total_est_cost:[450, 560, 670]
                # We will find path with smallest total_est_cost
    min_total_index = 0  # keep track of the index of of the min value of total_est_cost
    frontier = start  # The nearest node to the goal in a path
    nodes_has_been_explored = [] # a node has been explored, should not to concidered for another path.
                                 # This is because the shortest path from the start node to this node has been included to the path list already.
    # continue searching next frontier untill the shortest path's(the path with minimum total_est_cost) frontier reach the goal.
    while frontier != goal:  
        
        count_neighbors = 0  # count explored neighbors
        for node_next in M.roads[frontier]: # search neighbours
            if node_next in nodes_has_been_explored:  
                count_neighbors +=1
                if count_neighbors == len(M.roads[frontier]):
                    #edge case: 
                    #if a path with frontier has all neighbors has been explored but not reach to the goal, we delete this path.
                    del sol_list["path"][min_total_index]
                    del sol_list["path_cost"][min_total_index]
                    del sol_list["total_est_cost"][min_total_index]
                continue
            else:
                # add a new path with every neighbor
                dist_frontier_next = dist(nodes[frontier], nodes[node_next])  # distance between the frontier node to the next neighbour
                current_path_cost = sol_list["path_cost"][min_total_index]  # path_cost from the start node to the frontier node.
                est_next =  dist(nodes[node_next], nodes[goal])

                # put the neighbor after the frontier
                new_path = sol_list["path"][min_total_index] + [node_next]
                sol_list["path"].append(new_path)
                sol_list["path_cost"].append(current_path_cost + dist_frontier_next)
                sol_list["total_est_cost"].append(current_path_cost + dist_frontier_next + est_next)
        # there is no reachable path found
        if len(sol_list["path"]) == 0:
            return -1
        if count_neighbors < len(M.roads[frontier]):  # new pathes added.
            # delete the original path before extend. because it is already included in new pathes
            nodes_has_been_explored.append(frontier)  # mark the frontier node as has been explored node.
            del sol_list["path"][min_total_index]
            del sol_list["path_cost"][min_total_index]
            del sol_list["total_est_cost"][min_total_index]
            
        #find the mimimum_total in the sol_list, refresh the frontier node.whatever any path reaches the goal
        min_total_index = sol_list["total_est_cost"].index(min(sol_list["total_est_cost"]))
        frontier = sol_list["path"][min_total_index][-1]
        
    #shortest path's(the path with minimum total_est_cost) frontier reach the goal.
    # we found the sortest path
    return sol_list["path"][min_total_index]

In [12]:
map_40 = load_map('map-40.pickle')
show_map(map_40)

In [13]:
path = shortest_path(map_40, 5, 34)
if path == [5, 16, 37, 12, 34]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

shortest path called
great! Your code works for these inputs!


### Testing your Code
If the code below produces no errors, your algorithm is behaving correctly. You are almost ready to submit! Before you submit, go through the following submission checklist:

**Submission Checklist**

1. Does my code pass all tests?
2. Does my code implement `A*` search and not some other search algorithm?
3. Do I use an **admissible heuristic** to direct search efforts towards the goal?
4. Do I use data structures which avoid unnecessarily slow lookups?

When you can answer "yes" to all of these questions, submit by pressing the Submit button in the lower right!

In [14]:
from test import test

test(shortest_path)

shortest path called
shortest path called
shortest path called
All tests pass! Congratulations!


In [15]:
shortest_path(map_10, 4, 9)

shortest path called


-1

In [16]:
show_map(map_40, start=8, goal=24, path=[8, 14, 16, 37, 12, 17, 10, 24])

# DataStructure-Project4-A_star-route-planner  

This is a route-planning algorithm like the one used in Google Maps to calculate the shortest path between two points on a map.  
I used the A* algorithm to find a path.  
A* algorithm has a base function to calculate the total estimated cost of a path, which helps us to avoid "greed search" on the map.  
total estimated cost f = g + h  
g: the path cost, the sum of distances between every node in the path.  
h: the heuristic estimation, estimation of the cost from the frontier to the goal.  
I chose the heuristic estimation h the distance between the frontier to the goal. This can keep the h is the smallest cost from the frontier to the goal.  
Because of this, we do not need to consider a path with a bigger total estimated cost, because there is no hope to get a smaller total estimated cost after searching on that path.  
This means we can only focus on the path which has the smallest total estimated cost.  
Therefore, we will run in a loop that always extends a path that has the smallest total estimated cost until the frontier of the chosen path reaches the goal.  

# Challenges of the program  

## 1. All neighbors of a frontier should be included in a new path.  
The first version I built is to extend a path with only one neighbor.  
How did I do is, find the only one neighbor node which has the shortest estimated path cost and continue the searching until reach the goal. 
This algorithm returns me a path with low cost so far, but not the shortest path.  
For example, When there is the shortest path to the goal but it includes a node that not chosen by a frontier during the searching, and the reason was its estimated path cost was higher than the other one which has been chosen.  

So I changed the algorithm to extend a path with all neighbors of the frontier.  
Every time the program will go and look at the dictionary `sol_list` which saves all paths that have been explored and choose the path with the lowest estimated path cost for the next explore.  
The frontier, the last node of the chosen path will search its neighbors and add new paths to the path list.  

## 2. Edge case of the goal is unreachable  
I decided to use a list `nodes_has_been_explored` to keep explored nodes for the case the goal is in an unreachable place.  
Every time the frontier has been processed, the program will append it to the list `nodes_has_been_explored`, and the next frontier will not consider its neighbor that included in the list `nodes_has_been_explored`.  
This list also shortens our searching time, because the shortest path from the start node to the node in `nodes_has_been_explored` has been found and included in the path list already, any other paths to the node should be ignored.  
If a frontier has no neighbors or its neighbors all included in `nodes_has_been_explored`, this means, this path we are considering has been reached its end, it will impossible to reach the goal. The program will delete the path from the path list.  
Finally, if there is no path in the path list to continue searching, we can say we have no path to the goal. The program will return -1 in this case.  

# Files description 
The progam code is included in student_code.py.
You can run test.py for testing.

I would like to include beleow data in this notebook, This is for preparing further programming in local.

In [ ]:
map10_intersections = {0: [0.7798606835438107, 0.6922727646627362],
                     1: [0.7647837074641568, 0.3252670836724646],
                     2: [0.7155217893995438, 0.20026498027300055],
                     3: [0.7076566826610747, 0.3278339270610988],
                     4: [0.8325506249953353, 0.02310946309985762],
                     5: [0.49016747075266875, 0.5464878695400415],
                     6: [0.8820353070895344, 0.6791919587749445],
                     7: [0.46247219371675075, 0.6258061621642713],
                     8: [0.11622158839385677, 0.11236327488812581],
                     9: [0.1285377678230034, 0.3285840695698353]}
map10_roads = [[7, 6, 5],
             [4, 3, 2],
             [4, 3, 1],
             [5, 4, 1, 2],
             [1, 2, 3],
             [7, 0, 3],
             [0],
             [0, 5],
             [9],
             [8]]

In [ ]:
map40_intersections = {0: [0.7801603911549438, 0.49474860768712914],
                     1: [0.5249831588690298, 0.14953665513987202],
                     2: [0.8085335344099086, 0.7696330846542071],
                     3: [0.2599134798656856, 0.14485659826020547],
                     4: [0.7353838928272886, 0.8089961609345658],
                     5: [0.09088671576431506, 0.7222846879290787],
                     6: [0.313999018186756, 0.01876171413125327],
                     7: [0.6824813442515916, 0.8016111783687677],
                     8: [0.20128789391122526, 0.43196344222361227],
                     9: [0.8551947714242674, 0.9011339078096633],
                     10: [0.7581736589784409, 0.24026772497187532],
                     11: [0.25311953895059136, 0.10321622277398101],
                     12: [0.4813859169876731, 0.5006237737207431],
                     13: [0.9112422509614865, 0.1839028760606296],
                     14: [0.04580558670435442, 0.5886703168399895],
                     15: [0.4582523173083307, 0.1735506267461867],
                     16: [0.12939557977525573, 0.690016328140396],
                     17: [0.607698913404794, 0.362322730884702],
                     18: [0.719569201584275, 0.13985272363426526],
                     19: [0.8860336256842246, 0.891868301175821],
                     20: [0.4238357358399233, 0.026771817842421997],
                     21: [0.8252497121120052, 0.9532681441921305],
                     22: [0.47415009287034726, 0.7353428557575755],
                     23: [0.26253385360950576, 0.9768234503830939],
                     24: [0.9363713903322148, 0.13022993020357043],
                     25: [0.6243437191127235, 0.21665962402659544],
                     26: [0.5572917679006295, 0.2083567880838434],
                     27: [0.7482655725962591, 0.12631654071213483],
                     28: [0.6435799740880603, 0.5488515965193208],
                     29: [0.34509802713919313, 0.8800306496459869],
                     30: [0.021423673670808885, 0.4666482714834408],
                     31: [0.640952694324525, 0.3232711412508066],
                     32: [0.17440205342790494, 0.9528527425842739],
                     33: [0.1332965908314021, 0.3996510641743197],
                     34: [0.583993110207876, 0.42704536740474663],
                     35: [0.3073865727705063, 0.09186645974288632],
                     36: [0.740625863119245, 0.68128520136847],
                     37: [0.3345284735051981, 0.6569436279895382],
                     38: [0.17972981733780147, 0.999395685828547],
                     39: [0.6315322816286787, 0.7311657634689946]}
map40_roads = [[36, 34, 31, 28, 17],
             [35, 31, 27, 26, 25, 20, 18, 17, 15, 6],
             [39, 36, 21, 19, 9, 7, 4],
             [35, 20, 15, 11, 6],
             [39, 36, 21, 19, 9, 7, 2],
             [32, 16, 14],
             [35, 20, 15, 11, 1, 3],
             [39, 36, 22, 21, 19, 9, 2, 4],
             [33, 30, 14],
             [36, 21, 19, 2, 4, 7],
             [31, 27, 26, 25, 24, 18, 17, 13],
             [35, 20, 15, 3, 6],
             [37, 34, 31, 28, 22, 17],
             [27, 24, 18, 10],
             [33, 30, 16, 5, 8],
             [35, 31, 26, 25, 20, 17, 1, 3, 6, 11],
             [37, 30, 5, 14],
             [34, 31, 28, 26, 25, 18, 0, 1, 10, 12, 15],
             [31, 27, 26, 25, 24, 1, 10, 13, 17],
             [21, 2, 4, 7, 9],
             [35, 26, 1, 3, 6, 11, 15],
             [2, 4, 7, 9, 19],
             [39, 37, 29, 7, 12],
             [38, 32, 29],
             [27, 10, 13, 18],
             [34, 31, 27, 26, 1, 10, 15, 17, 18],
             [34, 31, 27, 1, 10, 15, 17, 18, 20, 25],
             [31, 1, 10, 13, 18, 24, 25, 26],
             [39, 36, 34, 31, 0, 12, 17],
             [38, 37, 32, 22, 23],
             [33, 8, 14, 16],
             [34, 0, 1, 10, 12, 15, 17, 18, 25, 26, 27, 28],
             [38, 5, 23, 29],
             [8, 14, 30],
             [0, 12, 17, 25, 26, 28, 31],
             [1, 3, 6, 11, 15, 20],
             [39, 0, 2, 4, 7, 9, 28],
             [12, 16, 22, 29],
             [23, 29, 32],
             [2, 4, 7, 22, 28, 36]]